<a href="https://colab.research.google.com/github/savinyu/VLG_Project/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.utils import compute_class_weight


In [6]:
train_df = pd.read_csv('train.csv')


In [7]:
X = train_df.drop('labels', axis=1).values
Y = train_df['labels'].values


X = X.reshape(X.shape[0], X.shape[1], 1)
X = X / 4


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

class_weights_dict = dict(enumerate(class_weights))

In [9]:

# Model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1200, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3, strides=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))

model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.45))

model.add(Dense(1, activation='sigmoid'))

# tf.keras.applications.ResNet50(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,

# )
# tf.keras.applications.VGG19(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )



model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [10]:
model.fit(X_train, y_train, batch_size=128, epochs=10, class_weight=class_weights_dict)



Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.3738
Epoch 2/10
1/1 [==============================] - 0s 141ms/step - loss: nan - accuracy: 0.7383
Epoch 3/10
1/1 [==============================] - 0s 149ms/step - loss: nan - accuracy: 0.7383
Epoch 4/10
1/1 [==============================] - 0s 141ms/step - loss: nan - accuracy: 0.7383
Epoch 5/10
1/1 [==============================] - 0s 162ms/step - loss: nan - accuracy: 0.7383
Epoch 6/10
1/1 [==============================] - 0s 133ms/step - loss: nan - accuracy: 0.7383
Epoch 7/10
1/1 [==============================] - 0s 143ms/step - loss: nan - accuracy: 0.7383
Epoch 8/10
1/1 [==============================] - 0s 137ms/step - loss: nan - accuracy: 0.7383
Epoch 9/10
1/1 [==============================] - 0s 155ms/step - loss: nan - accuracy: 0.7383
Epoch 10/10
1/1 [==============================] - 0s 161ms/step - loss: nan - accuracy: 0.7383


In [11]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

1/1 [==============================] - 0s 172ms/step
F1 score: 0.0


In [12]:
test_df = pd.read_csv('test.csv')


In [13]:
test = test_df.drop('id', axis=1).values
test = test.reshape(test.shape[0], test.shape[1], 1)

In [14]:
pred_probs = model.predict(test)
predicted_labels = (pred_probs > 0.5).astype(int)
predicted_labels = predicted_labels.reshape(predicted_labels.shape[0])

6/6 [==============================] - 0s 16ms/step


In [19]:
import csv
output_data = []
for i, value in enumerate(predicted_labels):
    output_data.append([i + 1, value])

output_file = 'output.csv'

with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['id', 'label'])
    writer.writerows(output_data)



print("CSV Output Done")


CSV Output Done


In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80        18
           1       0.00      0.00      0.00         9

    accuracy                           0.67        27
   macro avg       0.33      0.50      0.40        27
weighted avg       0.44      0.67      0.53        27



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
